# Import statements


In [2]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files

# Pokemon Database: pokedex

In [2]:
pokedex_url = requests.get("https://pokemondb.net/pokedex/all")
pokedex_soup = BeautifulSoup(pokedex_url.text, 'html.parser')

## Find table element

In [3]:
table = pokedex_soup.find('table')

## List of all cols and data
Approach: I will find all the information in each column and add the columns to a pandas dataframe

In [4]:
table_body = table.find('tbody')

Num list

In [5]:
pokemon_numbers_html = table_body.find_all('span', class_='infocard-cell-data')
pokemon_numbers = []
for num in pokemon_numbers_html:
  pokemon_numbers.append(num.contents[0])

Name list

In [6]:
pokemon_names_html = table_body.find_all('td', class_='cell-name')
pokemon_names = []
for data in pokemon_names_html:
  if None != data.find('small', class_='text-muted'):
    pokemon_names.append(data.find('small', class_='text-muted').contents[0])
  else:
    pokemon_names.append(data.find('a', class_='ent-name').contents[0])

Types

In [7]:
pokemon_types_html = table_body.find_all('td', class_='cell-icon')
type_1 = []
type_2 = []
for data in pokemon_types_html:
  type_1.append(data.find_all('a', class_='type-icon')[0].contents[0])
  try:
    type_2.append(data.find_all('a', class_='type-icon')[1].contents[0])
  except IndexError:
    type_2.append(None)

Stats

In [8]:
total_stats = []
hp = []
attack = []
defense = []
sp_attack = []
sp_defense = []
speed = []
for row in table_body.find_all('tr'):
  row_data = row.find_all('td', class_='cell-num')
  total_stats.append(row_data[1].contents[0]) # index 1 becasue the first td element with class_='cell-num' is the pokemon number and total_stats is the 2nd element of row_data
  hp.append(row_data[2].contents[0]) # Each following index relates to the stats position in the list row_data
  attack.append(row_data[3].contents[0])
  defense.append(row_data[4].contents[0])
  sp_attack.append(row_data[5].contents[0])
  sp_defense.append(row_data[6].contents[0])
  speed.append(row_data[7].contents[0])

Images

In [34]:
pokemon_image_html = table_body.find_all('span', class_='infocard-cell-img')
pokemon_sprite_urls = []
for img in pokemon_image_html:
  pokemon_sprite_urls.append(img.find('img').get('src'))


## Dataframe Creation

In [31]:
data = {
  'Sprite URL' : pokemon_sprite_urls,
  'Number' : pokemon_numbers,
  'Name': pokemon_names,
  'Type_1': type_1,
  'Type_2': type_2,
  'BST': total_stats,
  'Hp': hp,
  'Attack': attack,
  'Defense': defense,
  'Sp_Attack': sp_attack,
  'Sp_Defense': sp_defense,
  'Speed': speed
}

pokemon_dataframe = pd.DataFrame(data)

In [32]:
pokemon_dataframe

,Sprite URL,Number,Name,Type_1,Type_2,BST,Hp,Attack,Defense,Sp_Attack,Sp_Defense,Speed
0,https://img.pokemondb.net/sprites/sword-shield...,0001,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,https://img.pokemondb.net/sprites/sword-shield...,0002,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,https://img.pokemondb.net/sprites/sword-shield...,0003,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,https://img.pokemondb.net/sprites/sword-shield...,0003,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,https://img.pokemondb.net/sprites/sword-shield...,0004,Charmander,Fire,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...
1190,https://img.pokemondb.net/sprites/scarlet-viol...,1006,Iron Valiant,Fairy,Fighting,590,74,130,90,120,60,116
1191,https://img.pokemondb.net/sprites/scarlet-viol...,1007,Koraidon,Fighting,Dragon,670,100,135,115,85,100,135
1192,https://img.pokemondb.net/sprites/scarlet-viol...,1008,Miraidon,Electric,Dragon,670,100,85,100,135,115,135
1193,https://img.pokemondb.net/sprites/scarlet-viol...,1009,Walking Wake,Water,Dragon,590,99,83,91,125,83,109


## Export data as csv

In [33]:
pokemon_dataframe.to_csv("pokemon_data.csv")
files.download("pokemon_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Scaping Abilities

In [3]:
abilites_url = requests.get('https://pokemondb.net/ability')
abilities_soup = BeautifulSoup(abilites_url.text, 'html.parser')

## Abilites to dataframe


In [88]:
abilities_table = abilities_soup.find('tbody')
abilities_table_trs = abilities_table.find_all('tr')
abilities_data = {
    'Ability name': [],
    'Number of Pokemon': [],
    'Description': [],
    'Generation Introduced': []
}
for row in abilities_table_trs:
  row_data = row.find_all('td')
  abilities_data['Ability name'].append(row_data[0].text)
  abilities_data['Number of Pokemon'].append(row_data[1].text)
  abilities_data['Description'].append(row_data[2].text)
  abilities_data['Generation Introduced'].append(row_data[3].text)

abilities_df = pd.DataFrame(abilities_data)

## Export data as CSV

In [89]:
abilities_df.to_csv("ablities.csv")
files.download("ablities.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Scraping Items

In [4]:
item_url = requests.get('https://pokemondb.net/item/all')
item_soup = BeautifulSoup(item_url.text, 'html.parser')


## Items to dataframe

In [97]:
item_data = {
    "Name": [],
    "Category": [],
    "Effect": []
}
item_table = item_soup.find('tbody')
item_table_rows = item_table.find_all('tr')
for row in item_table_rows:
  row_data = row.find_all('td')
  item_data["Name"].append((row_data[0].text).strip())
  if row_data[1].text == '':
    item_data['Category'].append("None")
  else:
    item_data['Category'].append(row_data[1].text)
  item_data["Effect"].append(row_data[2].text)

item_dataframe = pd.DataFrame(item_data)
item_dataframe

,Name,Category,Effect
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...
2,Ability Shield,None,An item to be held by a Pokémon. This cute and...
3,Ability Urge,Battle items,"When used, it activates the Ability of an ally..."
4,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.
...,...,...,...
1439,Zangoose Claw,General items,Material obtained after defeating/catching Zan...
1440,Zap Plate,Hold items,Increases power of Electric-type moves. Change...
1441,Zinc,Medicine,Increases Special Defense EVs by 10.
1442,Zoom Lens,Hold items,Raises a move's accuracy if the holder moves a...


## Export to CSV

In [98]:
item_dataframe.to_csv('pokemon_items.csv')
files.download('pokemon_items.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Scraping Moves

In [5]:
moves_url = requests.get('https://pokemondb.net/move/all')
move_soup = BeautifulSoup(moves_url.text, 'html.parser')

## Moves to Dataframe

In [16]:
moves_data = {
    "Name": [],
    "Type": [],
    "Category": [],
    "Power": [],
    "Accuracy": [],
    "PP": [],
    "Effect": [],
    "Effect Chance": []
}
moves_table = move_soup.find('tbody')
move_table_rows = moves_table.find_all('tr')

for row in move_table_rows:
  row_data = row.find_all('td')
  moves_data["Name"].append(row_data[0].text)
  moves_data["Type"].append(row_data[1].text)
  if row_data[2].find('img') == None:
    moves_data['Category'].append("N\A")
  else:
    moves_data['Category'].append(row_data[2].find('img').get('alt'))
  moves_data["Power"].append(row_data[3].text)
  moves_data['Accuracy'].append(row_data[4].text)
  moves_data['PP'].append(row_data[5].text)
  if row_data[6].text == '':
    moves_data['Effect'].append('None')
  else:
    moves_data['Effect'].append(row_data[6].text)
  moves_data['Effect Chance'].append(row_data[7].text)

moves_dataframe = pd.DataFrame(moves_data)

## Export to CSV

In [17]:
moves_dataframe.to_csv('pokemon_moves.csv')
files.download('pokemon_moves.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>